In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
data = load_breast_cancer()
X = data.data

y = data.target

In [13]:
#print(X)

#print(y)

print(X.shape)
print(y.shape)


(569, 30)
(569,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)